In [1]:
import ROOT
from rootpy.tree import Tree, TreeModel, FloatCol, IntCol
from rootpy.io import root_open
from ROOT import gROOT, TCanvas, TF1, TFile, TTree, gRandom, TH1F

from ROOT import RooRealVar, RooFormulaVar, RooVoigtian, RooChebychev, RooArgList, \
                 RooArgSet, RooAddPdf, RooDataSet, RooCategory, RooSimultaneous, \
                 RooBreitWigner, RooCBShape, RooFFTConvPdf, RooGaussian,RooExponential, \
                 RooBinning, kRed, kBlue, kDotted,TString,RooAbsData, RooPlot, TCut, RooAbsData

In [2]:
import sys
sys.path.append("/home/chasenberg/repos/rootnotes")

import rootnotes

# For debugging only
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import scipy 
import root_pandas as rp
import root_numpy as ry 

import sys
sys.path.append('/home/chasenberg/dopy')
sys.path.append("/home/chasenberg/rootnotes")

import dopy.plotting as dpp
import dopy.sklearn_utils as sku


import matplotlib.pyplot as plt
%matplotlib inline
import rootnotes
from scipy.stats import chisquare
from root_numpy import root2array, tree2array, tree2array
from rootpy.io import root_open
import dopy.sklearn_utils as sku
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (5, 3)

/opt/rh/miniconda/envs/py3root6/lib/python3.4/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
branch_names_mc = ['B0_FitDaughtersConst*','idxPV','B0_L0*','B0_Hlt*','B0_BKGCAT','B0_FitPVConst_status','HLT*','J_psi_1S_Hlt*','J_psi_1S_L0*']

In [4]:
#Apply Cuts to dataframe: B0_FITDaughtersConst==0 & B0_FITPVConst==0 & idxPV=0 & B0_BKGCat==0 or B0_BKGCat==50

In [5]:
cut_string_mc = 'B0_FitDaughtersConst_status==0&B0_FitPVConst_status==0&idxPV==0&B0_BKGCAT==0'

In [6]:
mc_dir = '/fhgfs/users/chasenberg/mc/2015/jpsiks/dimuon/flattened/Bd2JpsiKS_dimuon_mc_2015_flat.root'

In [7]:
df_mc = rp.read_root(mc_dir,key='Bd2JpsiKs',columns=branch_names_mc,where=cut_string_mc,flatten=True)

In [8]:
df_mc = df_mc.replace([np.inf, -np.inf], np.nan)
df_mc = df_mc.dropna()
df_mc_evt = len(df_mc)
print('Number of events in MC: ',df_mc.shape[0])

Number of events in MC:  691974


In [9]:
#first create dataframe with required triggerlines and get rid of 'physics' and 'global'
'''df1 = df_mc.filter(regex=("B0_Hlt.*"))
df2 = df_mc.filter(regex=("B0_L0.*"))
df3 = df_mc.filter(regex=("J_psi_1S_Hlt.*"))
df4 = df_mc.filter(regex=("J_psi_1S_L0.*"))
df_mc = pd.concat([df1,df2,df3,df4],axis=1)'''
df_mc = df_mc.filter(regex=(".*TOS"))
df_forward = df_mc.filter(regex=(".*ForwardDecision"))
df_phys = df_mc.filter(regex=(".*Phys"))
#df_global = df_mc.filter(regex=(".*Global"))
triggerlines_phys = list(df_phys.columns.values)
#triggerlines_global = list(df_global.columns.values)
triggerlines_forward = list(df_forward.columns.values)
unwanted_triggers = np.concatenate((triggerlines_phys,triggerlines_forward),axis=0)#triggerlines_global
print("Just got rid of these triggerlines:")
print("------------------------------------")
for unwanted_trigger in unwanted_triggers:
    del df_mc[unwanted_trigger]
    print(unwanted_trigger)

Just got rid of these triggerlines:
------------------------------------
B0_L0PhysicsDecision_TOS
B0_Hlt1Phys_TOS
B0_Hlt2Phys_TOS
J_psi_1S_Hlt1Phys_TOS
J_psi_1S_Hlt2Phys_TOS
J_psi_1S_L0PhysicsDecision_TOS
B0_Hlt2ForwardDecision_TOS
J_psi_1S_Hlt2ForwardDecision_TOS


In [11]:
#Quick test
cut = 'B0_L0Global_TOS==1|B0_Hlt1DiMuonHighMassDecision_TOS|B0_Hlt2DiMuonDetachedJPsiDecision_TOS==1'
test = df_mc.query(cut)
efficiency = test.shape[0]/df_mc.shape[0]
print(efficiency)

0.8651293256683055


In [12]:
#Now look at the L0 lines

In [13]:
df_mc_L0 = df_mc.filter(regex=("L0.*"))
df_mc_L0.head(n=5)

,B0_L0Global_TOS,B0_L0MuonDecision_TOS,B0_L0DiMuonDecision_TOS,B0_L0MuonHighDecision_TOS,B0_L0HadronDecision_TOS,B0_L0ElectronDecision_TOS,B0_L0PhotonDecision_TOS,B0_Hlt1MultiMuonNoL0Decision_TOS,B0_Hlt1DiMuonNoL0Decision_TOS,B0_Hlt1L0AnyDecision_TOS,...,J_psi_1S_Hlt1L0AnyNoSPDDecision_TOS,J_psi_1S_Hlt2RadiativeLb2L0GammaEELLDecision_TOS,J_psi_1S_Hlt2RadiativeLb2L0GammaLLDecision_TOS,J_psi_1S_L0Global_TOS,J_psi_1S_L0MuonDecision_TOS,J_psi_1S_L0DiMuonDecision_TOS,J_psi_1S_L0MuonHighDecision_TOS,J_psi_1S_L0HadronDecision_TOS,J_psi_1S_L0ElectronDecision_TOS,J_psi_1S_L0PhotonDecision_TOS
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,True,True,False,False,False,False,False,False,False,False,...,False,False,False,True,True,False,False,False,False,False
2,True,True,False,False,False,False,False,False,False,False,...,False,False,False,True,True,False,False,False,False,False
3,True,True,False,False,False,False,False,False,False,False,...,False,False,False,True,True,False,False,False,False,False
4,True,True,True,False,False,False,False,False,True,False,...,False,False,False,True,True,True,False,False,False,False


In [14]:
trigger_L0 = list(df_mc_L0.columns.values)

In [15]:
#Loop of triggers and calculate efficiencies
trigger_combine = []
cutstring = np.str()
for i in range(len(trigger_L0)):
    a = df_mc_L0[trigger_L0[i]].count()
    b = df_mc_L0[df_mc_L0[trigger_L0[i]] == 1].count()
    efficiency = b/a
    #print(efficiency[1])
    if efficiency[1]>0.5:
        #print(trigger_L0[i],efficiency[1])
        cutstring = cutstring + '|' + trigger_L0[i]+'==1'
        trigger_combine.append(trigger_L0[i])
        #trigger_combine.append(trigger_L0[i])
        #efficiencies.append(efficiency[1])
selection_L0 = cutstring.lstrip('|')
print("The selected triggerlines are:",trigger_combine)
efficiency = df_mc_L0.query(selection_L0).shape[0] / df_mc_L0.shape[0] 
print('Efficiency of the combined cut is:',efficiency)        

The selected triggerlines are: ['B0_L0Global_TOS', 'B0_L0MuonDecision_TOS', 'B0_L0DiMuonDecision_TOS', 'J_psi_1S_L0Global_TOS', 'J_psi_1S_L0MuonDecision_TOS', 'J_psi_1S_L0DiMuonDecision_TOS']
Efficiency of the combined cut is: 0.821153684965042


In [16]:
#Now HLT1

In [17]:
df_mc_hlt1 = df_mc.query(selection_L0).filter(regex=("Hlt1.*"))

In [18]:
#Apply cut of the previous step

In [19]:
trigger_hlt1 = list(df_mc_hlt1.columns.values)

In [20]:
#Loop of triggers and calculate efficiencies
trigger_combine = []
cutstring = np.str()
for i in range(len(trigger_hlt1)):
    a = df_mc_hlt1[trigger_hlt1[i]].count()
    b = df_mc_hlt1[df_mc_hlt1[trigger_hlt1[i]] == 1].count()
    efficiency = b/a
    #print(efficiency[1])
    if efficiency[1]>0.9:
        #print(trigger_L0[i],efficiency[1])
        cutstring = cutstring + '|' + trigger_hlt1[i]+'==1'
        trigger_combine.append(trigger_hlt1[i])
        #trigger_combine.append(trigger_L0[i])
        #efficiencies.append(efficiency[1])
selection_hlt1 = cutstring.lstrip('|')
print("The selected triggerlines are:",trigger_combine)
print('The cutstring is:',selection_hlt1)
efficiency = df_mc_hlt1.query(selection_hlt1).shape[0] / df_mc_hlt1.shape[0] 
global_efficiency = df_mc_hlt1.query(selection_hlt1).shape[0] / df_mc.shape[0] 
print('Efficiency of the combined cut is:',efficiency)
print('Global efficiency of the combined cut is:',global_efficiency)

The selected triggerlines are: ['B0_Hlt1Global_TOS', 'B0_Hlt1CalibHighPTLowMultTrksDecision_TOS', 'J_psi_1S_Hlt1Global_TOS', 'J_psi_1S_Hlt1CalibHighPTLowMultTrksDecision_TOS']
The cutstring is: B0_Hlt1Global_TOS==1|B0_Hlt1CalibHighPTLowMultTrksDecision_TOS==1|J_psi_1S_Hlt1Global_TOS==1|J_psi_1S_Hlt1CalibHighPTLowMultTrksDecision_TOS==1
Efficiency of the combined cut is: 0.9954893992963956
Global efficiency of the combined cut is: 0.8174497885758714


In [21]:
check = df_mc_hlt1.query(selection_hlt1).shape[0] / df_mc.shape[0]
check

0.8174497885758714

In [22]:
#Now HLT2

In [23]:
#check
check1 = df_mc.query(selection_L0)
#check1 = check1.query(selection_hlt2)
#check1 = check1.query(selection_hlt2)
check = check1.shape[0] / df_mc.shape[0]
check

0.821153684965042

In [24]:
df_mc_hlt2 = df_mc.query(selection_hlt1)
df_mc_hlt2 = df_mc_hlt2.query(selection_L0).filter(regex=("Hlt2.*"))

In [25]:
#Apply cuts of the previous steps --> L0 and HLT1

In [26]:
trigger_hlt2 = list(df_mc_hlt2.columns.values)

In [27]:
#Loop of triggers and calculate efficiencies
trigger_combine = []
cutstring = np.str()
for i in range(len(trigger_hlt2)):
    a = df_mc_hlt2[trigger_hlt2[i]].count()
    b = df_mc_hlt2[df_mc_hlt2[trigger_hlt2[i]] == 1].count()
    efficiency = b/a
    #print(efficiency[1])
    if efficiency[1]>0.95:
        #print(trigger_L0[i],efficiency[1])
        cutstring = cutstring + '|' + trigger_hlt2[i]+'==1'
        trigger_combine.append(trigger_hlt2[i])
        #trigger_combine.append(trigger_L0[i])
        #efficiencies.append(efficiency[1])
selection_hlt2 = cutstring.lstrip('|')
print("The selected triggerlines are:",trigger_combine)
#print('The cutstring is:',selection_hlt2)
efficiency = df_mc_hlt2.query(selection_hlt2).shape[0] / df_mc_hlt2.shape[0] 
global_efficiency = df_mc_hlt2.query(selection_hlt2).shape[0] / df_mc.shape[0] 
print('Efficiency of the combined cut is:',efficiency)
print('Global efficiency of the combined cut is:',global_efficiency)

The selected triggerlines are: ['B0_Hlt2Global_TOS', 'B0_Hlt2DiMuonDetachedJPsiDecision_TOS', 'B0_Hlt2DiMuonJPsiDecision_TOS', 'J_psi_1S_Hlt2Global_TOS', 'J_psi_1S_Hlt2DiMuonDetachedJPsiDecision_TOS', 'J_psi_1S_Hlt2DiMuonJPsiDecision_TOS']
Efficiency of the combined cut is: 1.0
Global efficiency of the combined cut is: 0.8174497885758714


In [28]:
check = df_mc_hlt2.shape[0] / df_mc.shape[0]
check

0.8174497885758714

In [29]:
#Final efficiencies after all three trigger stages

In [30]:
df_mc_final = df_mc.query( selection_L0 and selection_hlt1 )#and selection_hlt2  )

In [31]:
efficiency = df_mc_final.shape[0] / df_mc.shape[0]
efficiency

0.8622477145095047

In [32]:
#Check efficiency for all combinations of triggerlines 

In [33]:
triggerlines = list(df_trigger.columns.values)
entries = df_trigger.shape[0]
for triggerline1 in triggerlines:
    for triggerline2 in triggerlines:
        if triggerline1 != triggerline2:
            test1 = df_trigger[(df_trigger[triggerline1] == 1)|(df_trigger[triggerline2] == 1)].shape[0]
            efficiency1 = test1/entries
            for triggerline3 in triggerlines:
                if (triggerline1 != triggerline3) and (triggerline2 != triggerline3):
                    test2 = df_trigger[(df_trigger[triggerline1] == 1)|(df_trigger[triggerline2] == 1)|(df_trigger[triggerline3] == 1)].shape[0]
                    efficiency2 = test2/entries
                    if (efficiency2 > 0.92) and (efficiency2>efficiency1):
                        print("Triggerline combination (3 triggers chosen):")
                        print("===========================================")
                        print(triggerline1,triggerline2,triggerline3)
                        print("The efficiency of these triggerlines combination is:",efficiency2)
                        print("---------------------------------------------------------------------------------------------------------------")
                    elif (efficiency1 > 0.92): 
                        print("Triggerline combination (2 triggers chosen):")
                        print("===========================================")
                        print(triggerline1,triggerline2)
                        print("The efficiency of these triggerlines combination is:",efficiency1)
                        print("---------------------------------------------------------------------------------------------------------------")

NameError: name 'df_trigger' is not defined